# Logistic Regression Neural Network Basics 
    Classifier
    Learn by loss and optimization functions
    The output is a class probability
    
    Goal is to predict which passengers will survive the titanic tragedy

In [2]:
import tensorflow as tf
import numpy as np 
import pandas as pd 
from pandas import DataFrame as DF, Series 


https://www.kaggle.com/c/titanic/data


**Variable	Definition	Key**

**survival**	Survival	0 = No, 1 = Yes
**pclass**	Ticket class	1 = 1st, 2 = 2nd, 3 = 3rd
**sex**	Sex	
**Age**	Age in years	
**sibsp**	# of siblings / spouses aboard the Titanic	
**parch**	# of parents / children aboard the Titanic	
**ticket**	Ticket number	
**fare**	Passenger fare	
**cabin**	Cabin number	
**embarked**	Port of Embarkation	C = Cherbourg, Q = Queenstown, S = Southampton


**Variable Notes**

pclass: A proxy for socio-economic status (SES)
1st = Upper
2nd = Middle
3rd = Lower

age: Age is fractional if less than 1. If the age is estimated, is it in the form of xx.5

sibsp: The dataset defines family relations in this way...
Sibling = brother, sister, stepbrother, stepsister
Spouse = husband, wife (mistresses and fiancés were ignored)

parch: The dataset defines family relations in this way...
Parent = mother, father
Child = daughter, son, stepdaughter, stepson
Some children travelled only with a nanny, therefore parch=0 for them.



In [19]:
data = pd.read_csv('TitanicTrain.csv')
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [15]:
data.drop(['PassengerId','Name','Ticket'], axis=1, inplace=True)

In [17]:
data.to_csv('data.csv',index=False)

#cleanup 
del data
import gc
gc.collect()

487

In [28]:
data = pd.read_csv('data.csv')

In [29]:
data.isnull().sum()

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Cabin       687
Embarked      2
dtype: int64

In [30]:
data.fillna({'Age':-1, 'Cabin':'Unk', 'Embarked':'Unk', 'Fare':-1},inplace=True)

**Very Basic - Hand coded Logistic Regression**

In [31]:
#TODO: we should use the other csv file for testing
#convert sex to binary 
data.loc[:,'Sex'] = (data.Sex == 'female').astype(int)

#Train/Test data split 
XTrain = data.loc[:,['Pclass','Sex','Age','SibSp','Parch','Fare']].sample(frac=0.75)
XTest  = data[~data.index.isin(XTrain.index)].loc[:,['Pclass','Sex','Age','SibSp','Parch','Fare']]

YTrain = pd.get_dummies(data[data.index.isin(XTrain.index)].Survived).values
YTest  = pd.get_dummies(data[~data.index.isin(XTrain.index)].Survived).values

**Create Model**

In [54]:
num_features = XTrain.shape[1] #number of features(observations)
num_classes = 2 #survived or not 

#None number of observation (features) should be flexible 
#it allows us to pass the batch size later on
X = tf.placeholder('float',[None, num_features])
Y = tf.placeholder('float',[None, num_classes])

# w = Weight array and b = bias array 
W = tf.Variable(tf.zeros([num_features, num_classes]))
B = tf.Variable(tf.zeros([num_classes]))

# define the logistic model 
# y = wx+b as a argument for the softmax 
# softmax cause when you have two outputs you end up with simple logistic probability 
#  wont sum to 1 the softmax will allow us to sum to 1
yhat = tf.nn.softmax(tf.matmul(X,W)+B)

# define the loss function
loss_fn = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=yhat,labels=Y))

# define optimizer that will minimize based on the loss function
opt = tf.train.AdamOptimizer(0.01).minimize(loss_fn)

# session 
session = tf.Session()

# init variables
init = tf.global_variables_initializer()
session.run(init)

num_epocs = 10

# loop throw the epocs and run the optimizer step on the full data each time 
for i in range(num_epocs):
    session.run(opt, feed_dict={X:XTrain,Y:YTrain})
    
# accuracy function
# argmax: Returns the **index** with the largest value across axes of a tensor (across the parameters)
# for every output we will compare the predection to our truth values
accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(yhat, 1), tf.argmax(Y, 1)), 'float'))

# get the test accuracy 
accuracy_value = session.run(accuracy, feed_dict={X:XTrain,Y:YTrain})
accuracy_value

#0.62 = 62% 

0.6287425

# ** Logistic Regression with Batching**

**Input function**

In [46]:
data = pd.read_csv('data.csv')
data.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,0,3,male,22.0,1,0,7.2500,NaN,S
1,1,1,female,38.0,1,0,71.2833,C85,C
2,1,3,female,26.0,0,0,7.9250,NaN,S
3,1,1,female,35.0,1,0,53.1000,C123,S
4,0,3,male,35.0,0,0,8.0500,NaN,S


In [48]:
data.Pclass.unique()

array(['male', 'female'], dtype=object)

In [49]:
data.Sex.unique()

array(['male', 'female'], dtype=object)

In [50]:
data.Cabin.unique()

array([nan, 'C85', 'C123', 'E46', 'G6', 'C103', 'D56', 'A6',
       'C23 C25 C27', 'B78', 'D33', 'B30', 'C52', 'B28', 'C83', 'F33',
       'F G73', 'E31', 'A5', 'D10 D12', 'D26', 'C110', 'B58 B60', 'E101',
       'F E69', 'D47', 'B86', 'F2', 'C2', 'E33', 'B19', 'A7', 'C49', 'F4',
       'A32', 'B4', 'B80', 'A31', 'D36', 'D15', 'C93', 'C78', 'D35',
       'C87', 'B77', 'E67', 'B94', 'C125', 'C99', 'C118', 'D7', 'A19',
       'B49', 'D', 'C22 C26', 'C106', 'C65', 'E36', 'C54',
       'B57 B59 B63 B66', 'C7', 'E34', 'C32', 'B18', 'C124', 'C91', 'E40',
       'T', 'C128', 'D37', 'B35', 'E50', 'C82', 'B96 B98', 'E10', 'E44',
       'A34', 'C104', 'C111', 'C92', 'E38', 'D21', 'E12', 'E63', 'A14',
       'B37', 'C30', 'D20', 'B79', 'E25', 'D46', 'B73', 'C95', 'B38',
       'B39', 'B22', 'C86', 'C70', 'A16', 'C101', 'C68', 'A10', 'E68',
       'B41', 'A20', 'D19', 'D50', 'D9', 'A23', 'B50', 'A26', 'D48',
       'E58', 'C126', 'B71', 'B51 B53 B55', 'D49', 'B5', 'B20', 'F G63',
       'C62 C64',

In [51]:
data.Embarked.unique()

array(['S', 'C', 'Q', nan], dtype=object)

In [55]:
# define columns and default values 
_csv_columns_defaults = [[0],[-1],['Unk'],[-1.],[0],[0],[-1.],['Unk'],['Unk']]
_csv_columns = data.columns.tolist()

# define the input function 
def input_fn(csv_file, features_names, batch_size=16, n_epocs=10, shuffle=False):
    def decode_csv(line):
        parsedLine = tf.decode_csv(line, _csv_columns_defaults)
        features_dict = dict(zip(features_names, parsedLine))
        labels = features_dict.pop('Survived') #remove Survived from dict 
        return features_dict, labels
    
    if shuffle:
        dataset = dataset.shuffle(buffer_size=100*1024) #100KB
        
    dataset = (tf.data.TextLineDataset(csv_file)
               .skip(1)
               .map(decode_csv, num_parallel_calls=3))
    dataset = dataset.batch(batch_size)
    dataset = dataset.repeat(n_epocs)
    itrator = dataset.make_one_shot_iterator()
    batch_feature, batch_labels = itrator.get_next()
    return batch_feature, batch_labels



### Handling Categorical Features

using  `tf.feature_colunm` is to map data to a model

**Base Categorical Features**

In [58]:
sex = tf.feature_column.categorical_column_with_vocabulary_list('Sex',vocabulary_list=['female','male','Unk'])
embarked = tf.feature_column.categorical_column_with_vocabulary_list('Embarked',vocabulary_list=['S','C','Q','Unk'])


**Base Continuous Features**

In [63]:
age = tf.feature_column.numeric_column('Age')
# you can also create upper level buckets like
# age_buckets = tf.feature_column.bucketized_column(age, boundaries=[5.,10,18,25,35,45,55,65])

sib = tf.feature_column.numeric_column('SibSp')
parch = tf.feature_column.numeric_column('Parch')
fere = tf.feature_column.numeric_column('Fare')


**Create Model**

In [66]:
columns = [age,sib,parch,fere,sex,embarked]
model_dir = 'lr_model'
# use the logistic resression (LinearClassifier)
model = tf.estimator.LinearClassifier(model_dir=model_dir, feature_columns=columns, optimizer=tf.train.AdamOptimizer())

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'lr_model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000000947DFF12B0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


**Train Model**

In [67]:
model.train(input_fn=lambda: input_fn('data.csv',_csv_columns))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into lr_model\model.ckpt.
INFO:tensorflow:loss = 11.090355, step = 1
INFO:tensorflow:global_step/sec: 418.12
INFO:tensorflow:loss = 9.610938, step = 101 (0.239 sec)
INFO:tensorflow:global_step/sec: 1086.19
INFO:tensorflow:loss = 12.629805, step = 201 (0.092 sec)
INFO:tensorflow:global_step/sec: 1051.88
INFO:tensorflow:loss = 8.880047, step = 301 (0.095 sec)
INFO:tensorflow:global_step/sec: 1051.88
INFO:tensorflow:loss = 9.075473, step = 401 (0.095 sec)
INFO:tensorflow:global_step/sec: 1040.93
INFO:tensorflow:loss = 6.975941, step = 501 (0.096 sec)
INFO:tensorflow:Saving checkpoints for 560 into lr_model\model.ckpt.
INFO:tensorflow:Loss for final step: 5.958311.


In [69]:
results = model.evaluate(input_fn=lambda: input_fn('data.csv',_csv_columns,n_epocs=1))
results

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-04-22-07:56:09
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from lr_model\model.ckpt-560
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-04-22-07:56:09
INFO:tensorflow:Saving dict for global step 560: accuracy = 0.7665544, accuracy_baseline = 0.6161616, auc = 0.8267983, auc_precision_recall = 0.7445791, average_loss = 0.5473142, global_step = 560, label/mean = 0.3838384, loss = 8.70816, prediction/mean = 0.4256734


{'accuracy': 0.7665544,
 'accuracy_baseline': 0.6161616,
 'auc': 0.8267983,
 'auc_precision_recall': 0.7445791,
 'average_loss': 0.5473142,
 'global_step': 560,
 'label/mean': 0.3838384,
 'loss': 8.70816,
 'prediction/mean': 0.4256734}